In [1]:
#import the libraries 

import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import fftpack,signal
import matplotlib
from matplotlib.ticker import ScalarFormatter
from astropy.visualization import (MinMaxInterval, SqrtStretch, ImageNormalize, ZScaleInterval)
import scipy
import json
import subprocess
import os

from exocrires import spectra_2d
from exocrires import analysis

# Run  the Color Analysis first to retrieve the Apparent Magnitude for Planets

In [2]:
#see eg_analysis.ipynb
mag_estimate=pd.read_csv('/Volumes/JCP_work/thesis_ms/2411/mag_estimate.csv')

# Process the Spectrum of the Host Star

In [3]:
data=spectra_2d.load_json('/Volumes/JCP_work/thesis_ms/input_1709_AO.json','r')

print(data['target']['brightness']['params']['mag'])


6.41


In [4]:
out_put=[]
for i in data['output']:
    if i == 'psf':
        out_put.append('%s'%i)
    else:   
        for j in data['output'][i]:
            out_put.append('%s_%s'%(i,j))
out_put=[string.lower() for string in out_put] 

In [5]:
simu_star=spectra_2d.spectra_2d(path_etc_local='/Volumes/JCP_work/thesis_ms/etc_cli.py', path_etc_input='/Volumes/JCP_work/thesis_ms/input_1709_AO.json', path_input_modi='/Volumes/JCP_work/thesis_ms/modi_input_1709_AO.json',\
             path_etc_output='/Volumes/JCP_work/thesis_ms/2411/star/', target='star', aperture_size=35, date='2411',\
        order_window=[[1921.318, 1961.128],[1989.978, 2031.165],[2063.711, 2106.392],[2143.087, 2187.386], [2228.786, 2274.835],\
        [2321.596, 2369.534], [2422.415, 2472.388]],\
        order_number=[29, 28, 27, 26, 25, 24, 23])

simu_star.output_json()

simu_star.json2ascii(path_json2ascii='/Volumes/JCP_work/thesis_ms/etc_json2ascii.py')

simu_star.ascii2txt(output=out_put)

data_log=simu_star.signal(detectors=3, contribution='target', focal_plane=(2048, 2048*3))

data_log.to_csv(simu_star.path_output+'stellar_simu.csv')

mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_1/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_3/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_5/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_7/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_9/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_11/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_13/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_15/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_17/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_19/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_21/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_23/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_25/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_27/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/star/ascii_29

the ETC calculation is done, the output file is saved at /Volumes/JCP_work/thesis_ms/2411/star/
File Not Found: 1-29-1-snr_noise_components
File Not Found: 1-29-1-maxsignals_snr
File Not Found: 1-29-1-maxsignals_noise_components
txt_done_1_29_1
File Not Found: 1-28-1-snr_noise_components
File Not Found: 1-28-1-maxsignals_snr
File Not Found: 1-28-1-maxsignals_noise_components
txt_done_1_28_1
File Not Found: 1-27-1-snr_noise_components
File Not Found: 1-27-1-maxsignals_snr
File Not Found: 1-27-1-maxsignals_noise_components
txt_done_1_27_1
File Not Found: 1-26-1-snr_noise_components
File Not Found: 1-26-1-maxsignals_snr
File Not Found: 1-26-1-maxsignals_noise_components
txt_done_1_26_1
File Not Found: 1-25-1-snr_noise_components
File Not Found: 1-25-1-maxsignals_snr
File Not Found: 1-25-1-maxsignals_noise_components
txt_done_1_25_1
File Not Found: 1-24-1-snr_noise_components
File Not Found: 1-24-1-maxsignals_snr
File Not Found: 1-24-1-maxsignals_noise_components
txt_done_1_24_1
File Not F

File Not Found: 5-23-2-maxsignals_snr
File Not Found: 5-23-2-maxsignals_noise_components
txt_done_5_23_2
File Not Found: 5-29-3-snr_noise_components
File Not Found: 5-29-3-maxsignals_snr
File Not Found: 5-29-3-maxsignals_noise_components
txt_done_5_29_3
File Not Found: 5-28-3-snr_noise_components
File Not Found: 5-28-3-maxsignals_snr
File Not Found: 5-28-3-maxsignals_noise_components
txt_done_5_28_3
File Not Found: 5-27-3-snr_noise_components
File Not Found: 5-27-3-maxsignals_snr
File Not Found: 5-27-3-maxsignals_noise_components
txt_done_5_27_3
File Not Found: 5-26-3-snr_noise_components
File Not Found: 5-26-3-maxsignals_snr
File Not Found: 5-26-3-maxsignals_noise_components
txt_done_5_26_3
File Not Found: 5-25-3-snr_noise_components
File Not Found: 5-25-3-maxsignals_snr
File Not Found: 5-25-3-maxsignals_noise_components
txt_done_5_25_3
File Not Found: 5-24-3-snr_noise_components
File Not Found: 5-24-3-maxsignals_snr
File Not Found: 5-24-3-maxsignals_noise_components
txt_done_5_24_3
F

File Not Found: 11-24-1-maxsignals_snr
File Not Found: 11-24-1-maxsignals_noise_components
txt_done_11_24_1
File Not Found: 11-23-1-snr_noise_components
File Not Found: 11-23-1-maxsignals_snr
File Not Found: 11-23-1-maxsignals_noise_components
txt_done_11_23_1
File Not Found: 11-29-2-snr_noise_components
File Not Found: 11-29-2-maxsignals_snr
File Not Found: 11-29-2-maxsignals_noise_components
txt_done_11_29_2
File Not Found: 11-28-2-snr_noise_components
File Not Found: 11-28-2-maxsignals_snr
File Not Found: 11-28-2-maxsignals_noise_components
txt_done_11_28_2
File Not Found: 11-27-2-snr_noise_components
File Not Found: 11-27-2-maxsignals_snr
File Not Found: 11-27-2-maxsignals_noise_components
txt_done_11_27_2
File Not Found: 11-26-2-snr_noise_components
File Not Found: 11-26-2-maxsignals_snr
File Not Found: 11-26-2-maxsignals_noise_components
txt_done_11_26_2
File Not Found: 11-25-2-snr_noise_components
File Not Found: 11-25-2-maxsignals_snr
File Not Found: 11-25-2-maxsignals_noise_co

File Not Found: 15-26-3-maxsignals_snr
File Not Found: 15-26-3-maxsignals_noise_components
txt_done_15_26_3
File Not Found: 15-25-3-snr_noise_components
File Not Found: 15-25-3-maxsignals_snr
File Not Found: 15-25-3-maxsignals_noise_components
txt_done_15_25_3
File Not Found: 15-24-3-snr_noise_components
File Not Found: 15-24-3-maxsignals_snr
File Not Found: 15-24-3-maxsignals_noise_components
txt_done_15_24_3
File Not Found: 15-23-3-snr_noise_components
File Not Found: 15-23-3-maxsignals_snr
File Not Found: 15-23-3-maxsignals_noise_components
txt_done_15_23_3
File Not Found: 17-29-1-snr_noise_components
File Not Found: 17-29-1-maxsignals_snr
File Not Found: 17-29-1-maxsignals_noise_components
txt_done_17_29_1
File Not Found: 17-28-1-snr_noise_components
File Not Found: 17-28-1-maxsignals_snr
File Not Found: 17-28-1-maxsignals_noise_components
txt_done_17_28_1
File Not Found: 17-27-1-snr_noise_components
File Not Found: 17-27-1-maxsignals_snr
File Not Found: 17-27-1-maxsignals_noise_co

File Not Found: 21-28-2-snr_noise_components
File Not Found: 21-28-2-maxsignals_snr
File Not Found: 21-28-2-maxsignals_noise_components
txt_done_21_28_2
File Not Found: 21-27-2-snr_noise_components
File Not Found: 21-27-2-maxsignals_snr
File Not Found: 21-27-2-maxsignals_noise_components
txt_done_21_27_2
File Not Found: 21-26-2-snr_noise_components
File Not Found: 21-26-2-maxsignals_snr
File Not Found: 21-26-2-maxsignals_noise_components
txt_done_21_26_2
File Not Found: 21-25-2-snr_noise_components
File Not Found: 21-25-2-maxsignals_snr
File Not Found: 21-25-2-maxsignals_noise_components
txt_done_21_25_2
File Not Found: 21-24-2-snr_noise_components
File Not Found: 21-24-2-maxsignals_snr
File Not Found: 21-24-2-maxsignals_noise_components
txt_done_21_24_2
File Not Found: 21-23-2-snr_noise_components
File Not Found: 21-23-2-maxsignals_snr
File Not Found: 21-23-2-maxsignals_noise_components
txt_done_21_23_2
File Not Found: 21-29-3-snr_noise_components
File Not Found: 21-29-3-maxsignals_sn

File Not Found: 27-29-1-snr_noise_components
File Not Found: 27-29-1-maxsignals_snr
File Not Found: 27-29-1-maxsignals_noise_components
txt_done_27_29_1
File Not Found: 27-28-1-snr_noise_components
File Not Found: 27-28-1-maxsignals_snr
File Not Found: 27-28-1-maxsignals_noise_components
txt_done_27_28_1
File Not Found: 27-27-1-snr_noise_components
File Not Found: 27-27-1-maxsignals_snr
File Not Found: 27-27-1-maxsignals_noise_components
txt_done_27_27_1
File Not Found: 27-26-1-snr_noise_components
File Not Found: 27-26-1-maxsignals_snr
File Not Found: 27-26-1-maxsignals_noise_components
txt_done_27_26_1
File Not Found: 27-25-1-snr_noise_components
File Not Found: 27-25-1-maxsignals_snr
File Not Found: 27-25-1-maxsignals_noise_components
txt_done_27_25_1
File Not Found: 27-24-1-snr_noise_components
File Not Found: 27-24-1-maxsignals_snr
File Not Found: 27-24-1-maxsignals_noise_components
txt_done_27_24_1
File Not Found: 27-23-1-snr_noise_components
File Not Found: 27-23-1-maxsignals_sn

File Not Found: 31-24-2-snr_noise_components
File Not Found: 31-24-2-maxsignals_snr
File Not Found: 31-24-2-maxsignals_noise_components
txt_done_31_24_2
File Not Found: 31-23-2-snr_noise_components
File Not Found: 31-23-2-maxsignals_snr
File Not Found: 31-23-2-maxsignals_noise_components
txt_done_31_23_2
File Not Found: 31-29-3-snr_noise_components
File Not Found: 31-29-3-maxsignals_snr
File Not Found: 31-29-3-maxsignals_noise_components
txt_done_31_29_3
File Not Found: 31-28-3-snr_noise_components
File Not Found: 31-28-3-maxsignals_snr
File Not Found: 31-28-3-maxsignals_noise_components
txt_done_31_28_3
File Not Found: 31-27-3-snr_noise_components
File Not Found: 31-27-3-maxsignals_snr
File Not Found: 31-27-3-maxsignals_noise_components
txt_done_31_27_3
File Not Found: 31-26-3-snr_noise_components
File Not Found: 31-26-3-maxsignals_snr
File Not Found: 31-26-3-maxsignals_noise_components
txt_done_31_26_3
File Not Found: 31-25-3-snr_noise_components
File Not Found: 31-25-3-maxsignals_sn

In [6]:
simu_star.plot_signal(wave_count_sort=data_log, max_percentile=99,\
                      save_path=simu_star.path_output+'star_simu.png', interv=ZScaleInterval())

# Process the Sky Background

In [7]:
# The sky background distribution should be the same for each run of output data. Therefore we only need to run it once with any input specgtrum of the planet

path='/Volumes/JCP_work/thesis_ms'

data_log_k=simu_star.signal(detectors=3, focal_plane=(2048, 2048*3), contribution='sky')
data_log_k.to_csv(path+'/2411/sky_noi/sky_simu.csv')
simu_star.plot_signal(data_log_k, max_percentile=99.99, save_path=path+'/2411/sky_noi/sky_simu.png', interv=ZScaleInterval(), Nor=None)

# Process the Spectrum of the Planet and Make Combination

In [8]:
#set up the array for contrast data
cont=np.zeros((55, 2, 2046*3))

#set up noise parameters
read=6 #e-/pix
dark=0.003 #e-/pix/s
dit=120
ndit=18
nspec=1
nspat=119

npix=nspec*nspat
ndark=dit*dark #e-/pix/exposure

#set up the angular separation (pix)
po=int(np.ceil(0.319/0.059))

In [9]:
data_pp=spectra_2d.load_json('/Volumes/JCP_work/thesis_ms/input_1709_p_AO.json','r')

data_pp['target']['brightness']['params']['mag']

19.077777944719617

In [10]:
out_put=[]
for i in data_pp['output']:
    if i == 'psf':
        out_put.append('%s'%i)
    else:   
        for j in data_pp['output'][i]:
            out_put.append('%s_%s'%(i,j))
out_put=[string.lower() for string in out_put] 

In [ ]:
path='/Volumes/JCP_work/thesis_ms'

filenames=glob.glob(path+'/models_1683734698/bt-settl_m/lte*.dat') #use dat
filenames=sorted(filenames)

m=6
for filename in filenames[m:]:
    
    #if os.path.exists(path+'/2411/planet/%s'%filename[-34:-23]) == True:
    #continue
    print ('now we start to process the model %s'%filename[-34:-23])
    
    data_pp['target']['brightness']['params']['mag']=mag_estimate.mag_ks_j[m]
    
    with open(path+'/input_1709_p_AO.json', 'w') as file:
        json.dump(data_pp, file)
        
    print ('apparent magnitude for the planet is taken as %s'%data_pp['target']['brightness']['params']['mag'])
    
    simu_planet=spectra_2d.spectra_2d(path_etc_local=path+'/etc_cli.py', path_etc_input=path+'/input_1709_p_AO.json', path_input_modi=path+'/modi_input_1709_p_AO.json',\
                 path_etc_output=path+'/2411/planet/%s/'%filename[-34:-23], target='planet', aperture_size=35, date='2411',\
            order_window=[[1921.318, 1961.128],[1989.978, 2031.165],[2063.711, 2106.392],[2143.087, 2187.386], [2228.786, 2274.835],\
            [2321.596, 2369.534], [2422.415, 2472.388]],\
            order_number=[29, 28, 27, 26, 25, 24, 23],\
            model_name=filename)

    simu_planet.output_json()

    simu_planet.json2ascii(path_json2ascii=path+'/etc_json2ascii.py')

    simu_planet.ascii2txt(output=out_put)

    data_log_p=simu_planet.signal(detectors=3, focal_plane=(2048, 2048*3), contribution='target')

    data_log_p.to_csv(simu_planet.path_output+'planet_simu.csv')
    
    print ('Planet-only 2d spectrum done:%s'%simu_planet.path_output)
    
    simu_planet.plot_signal(data_log_p, max_percentile=99,\
                            save_path=simu_planet.path_output+'planet_simu.png', interv=ZScaleInterval())
    
    ##-----Combine with the sky background, the stellar light and the noise terms-----##
    
    tot_data_com, fig=simu_planet.combine(data_star=data_log, data_sky=data_log_k, data_p=data_log_p, ron=read, dark=ndark, d=po, \
                                focal_plane=(2048,2048*3), plot_combination=True)
    fig.savefig(path+'/2411/planet/%s/combine.png'%filename[-34:-23])

    np.save(path+'/2411/planet/%s/combine_focal_plane.npy'%filename[-34:-23], tot_data_com)
    
    m+=1

now we start to process the model 011-4.5-0.0
apparent magnitude for the planet is taken as 19.11453211860835


mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_1/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_3/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_5/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_7/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_9/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_11/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_13/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_15/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_17/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_19/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_21/: File exists
mkdir: /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/ascii_23/: File exists
mkdir: /Volumes/JCP_w

the ETC calculation is done, the output file is saved at /Volumes/JCP_work/thesis_ms/2411/planet/011-4.5-0.0/
File Not Found: 1-29-1-snr_noise_components
File Not Found: 1-29-1-maxsignals_snr
File Not Found: 1-29-1-maxsignals_noise_components
txt_done_1_29_1
File Not Found: 1-28-1-snr_noise_components
File Not Found: 1-28-1-maxsignals_snr
File Not Found: 1-28-1-maxsignals_noise_components
txt_done_1_28_1
File Not Found: 1-27-1-snr_noise_components
File Not Found: 1-27-1-maxsignals_snr
File Not Found: 1-27-1-maxsignals_noise_components
txt_done_1_27_1
File Not Found: 1-26-1-snr_noise_components
File Not Found: 1-26-1-maxsignals_snr
File Not Found: 1-26-1-maxsignals_noise_components
txt_done_1_26_1
File Not Found: 1-25-1-snr_noise_components
File Not Found: 1-25-1-maxsignals_snr
File Not Found: 1-25-1-maxsignals_noise_components
txt_done_1_25_1
File Not Found: 1-24-1-snr_noise_components
File Not Found: 1-24-1-maxsignals_snr
File Not Found: 1-24-1-maxsignals_noise_components
txt_done_1_2

File Not Found: 5-24-2-maxsignals_snr
File Not Found: 5-24-2-maxsignals_noise_components
txt_done_5_24_2
File Not Found: 5-23-2-snr_noise_components
File Not Found: 5-23-2-maxsignals_snr
File Not Found: 5-23-2-maxsignals_noise_components
txt_done_5_23_2
File Not Found: 5-29-3-snr_noise_components
File Not Found: 5-29-3-maxsignals_snr
File Not Found: 5-29-3-maxsignals_noise_components
txt_done_5_29_3
File Not Found: 5-28-3-snr_noise_components
File Not Found: 5-28-3-maxsignals_snr
File Not Found: 5-28-3-maxsignals_noise_components
txt_done_5_28_3
File Not Found: 5-27-3-snr_noise_components
File Not Found: 5-27-3-maxsignals_snr
File Not Found: 5-27-3-maxsignals_noise_components
txt_done_5_27_3
File Not Found: 5-26-3-snr_noise_components
File Not Found: 5-26-3-maxsignals_snr
File Not Found: 5-26-3-maxsignals_noise_components
txt_done_5_26_3
File Not Found: 5-25-3-snr_noise_components
File Not Found: 5-25-3-maxsignals_snr
File Not Found: 5-25-3-maxsignals_noise_components
txt_done_5_25_3
F

File Not Found: 11-25-1-maxsignals_snr
File Not Found: 11-25-1-maxsignals_noise_components
txt_done_11_25_1
File Not Found: 11-24-1-snr_noise_components
File Not Found: 11-24-1-maxsignals_snr
File Not Found: 11-24-1-maxsignals_noise_components
txt_done_11_24_1
File Not Found: 11-23-1-snr_noise_components
File Not Found: 11-23-1-maxsignals_snr
File Not Found: 11-23-1-maxsignals_noise_components
txt_done_11_23_1
File Not Found: 11-29-2-snr_noise_components
File Not Found: 11-29-2-maxsignals_snr
File Not Found: 11-29-2-maxsignals_noise_components
txt_done_11_29_2
File Not Found: 11-28-2-snr_noise_components
File Not Found: 11-28-2-maxsignals_snr
File Not Found: 11-28-2-maxsignals_noise_components
txt_done_11_28_2
File Not Found: 11-27-2-snr_noise_components
File Not Found: 11-27-2-maxsignals_snr
File Not Found: 11-27-2-maxsignals_noise_components
txt_done_11_27_2
File Not Found: 11-26-2-snr_noise_components
File Not Found: 11-26-2-maxsignals_snr
File Not Found: 11-26-2-maxsignals_noise_co

...

File Not Found: 11-28-3-maxsignals_snr
File Not Found: 11-28-3-maxsignals_noise_components
txt_done_11_28_3
File Not Found: 11-27-3-snr_noise_components
File Not Found: 11-27-3-maxsignals_snr
File Not Found: 11-27-3-maxsignals_noise_components
txt_done_11_27_3
File Not Found: 11-26-3-snr_noise_components
File Not Found: 11-26-3-maxsignals_snr
File Not Found: 11-26-3-maxsignals_noise_components
txt_done_11_26_3
File Not Found: 11-25-3-snr_noise_components
File Not Found: 11-25-3-maxsignals_snr
File Not Found: 11-25-3-maxsignals_noise_components
txt_done_11_25_3
File Not Found: 11-24-3-snr_noise_components
File Not Found: 11-24-3-maxsignals_snr
File Not Found: 11-24-3-maxsignals_noise_components
txt_done_11_24_3
File Not Found: 11-23-3-snr_noise_components
File Not Found: 11-23-3-maxsignals_snr
File Not Found: 11-23-3-maxsignals_noise_components
txt_done_11_23_3
File Not Found: 13-29-1-snr_noise_components
File Not Found: 13-29-1-maxsignals_snr
File Not Found: 13-29-1-maxsignals_noise_co

File Not Found: 17-23-1-maxsignals_snr
File Not Found: 17-23-1-maxsignals_noise_components
txt_done_17_23_1
File Not Found: 17-29-2-snr_noise_components
File Not Found: 17-29-2-maxsignals_snr
File Not Found: 17-29-2-maxsignals_noise_components
txt_done_17_29_2
File Not Found: 17-28-2-snr_noise_components
File Not Found: 17-28-2-maxsignals_snr
File Not Found: 17-28-2-maxsignals_noise_components
txt_done_17_28_2
File Not Found: 17-27-2-snr_noise_components
File Not Found: 17-27-2-maxsignals_snr
File Not Found: 17-27-2-maxsignals_noise_components
txt_done_17_27_2
File Not Found: 17-26-2-snr_noise_components
File Not Found: 17-26-2-maxsignals_snr
File Not Found: 17-26-2-maxsignals_noise_components
txt_done_17_26_2
File Not Found: 17-25-2-snr_noise_components
File Not Found: 17-25-2-maxsignals_snr
File Not Found: 17-25-2-maxsignals_noise_components
txt_done_17_25_2
File Not Found: 17-24-2-snr_noise_components
File Not Found: 17-24-2-maxsignals_snr
File Not Found: 17-24-2-maxsignals_noise_co

File Not Found: 21-25-3-maxsignals_snr
File Not Found: 21-25-3-maxsignals_noise_components
txt_done_21_25_3
File Not Found: 21-24-3-snr_noise_components
File Not Found: 21-24-3-maxsignals_snr
File Not Found: 21-24-3-maxsignals_noise_components
txt_done_21_24_3
File Not Found: 21-23-3-snr_noise_components
File Not Found: 21-23-3-maxsignals_snr
File Not Found: 21-23-3-maxsignals_noise_components
txt_done_21_23_3
File Not Found: 23-29-1-snr_noise_components
File Not Found: 23-29-1-maxsignals_snr
File Not Found: 23-29-1-maxsignals_noise_components
txt_done_23_29_1
File Not Found: 23-28-1-snr_noise_components
File Not Found: 23-28-1-maxsignals_snr
File Not Found: 23-28-1-maxsignals_noise_components
txt_done_23_28_1
File Not Found: 23-27-1-snr_noise_components
File Not Found: 23-27-1-maxsignals_snr
File Not Found: 23-27-1-maxsignals_noise_components
txt_done_23_27_1
File Not Found: 23-26-1-snr_noise_components
File Not Found: 23-26-1-maxsignals_snr
File Not Found: 23-26-1-maxsignals_noise_co

File Not Found: 27-27-2-snr_noise_components
File Not Found: 27-27-2-maxsignals_snr
File Not Found: 27-27-2-maxsignals_noise_components
txt_done_27_27_2
File Not Found: 27-26-2-snr_noise_components
File Not Found: 27-26-2-maxsignals_snr
File Not Found: 27-26-2-maxsignals_noise_components
txt_done_27_26_2
File Not Found: 27-25-2-snr_noise_components
File Not Found: 27-25-2-maxsignals_snr
File Not Found: 27-25-2-maxsignals_noise_components
txt_done_27_25_2
File Not Found: 27-24-2-snr_noise_components
File Not Found: 27-24-2-maxsignals_snr
File Not Found: 27-24-2-maxsignals_noise_components
txt_done_27_24_2
File Not Found: 27-23-2-snr_noise_components
File Not Found: 27-23-2-maxsignals_snr
File Not Found: 27-23-2-maxsignals_noise_components
txt_done_27_23_2
File Not Found: 27-29-3-snr_noise_components
File Not Found: 27-29-3-maxsignals_snr
File Not Found: 27-29-3-maxsignals_noise_components
txt_done_27_29_3
File Not Found: 27-28-3-snr_noise_components
File Not Found: 27-28-3-maxsignals_sn

File Not Found: 33-29-1-snr_noise_components
File Not Found: 33-29-1-maxsignals_snr
File Not Found: 33-29-1-maxsignals_noise_components
txt_done_33_29_1
File Not Found: 33-28-1-snr_noise_components
File Not Found: 33-28-1-maxsignals_snr
File Not Found: 33-28-1-maxsignals_noise_components
txt_done_33_28_1
File Not Found: 33-27-1-snr_noise_components
File Not Found: 33-27-1-maxsignals_snr
File Not Found: 33-27-1-maxsignals_noise_components
txt_done_33_27_1
File Not Found: 33-26-1-snr_noise_components
File Not Found: 33-26-1-maxsignals_snr
File Not Found: 33-26-1-maxsignals_noise_components
txt_done_33_26_1
File Not Found: 33-25-1-snr_noise_components
File Not Found: 33-25-1-maxsignals_snr
File Not Found: 33-25-1-maxsignals_noise_components
txt_done_33_25_1
File Not Found: 33-24-1-snr_noise_components
File Not Found: 33-24-1-maxsignals_snr
File Not Found: 33-24-1-maxsignals_noise_components
txt_done_33_24_1
File Not Found: 33-23-1-snr_noise_components
File Not Found: 33-23-1-maxsignals_sn

In [9]:
#plot the contrast and save the average value
path='/Volumes/JCP_work/thesis_ms'

filenames=glob.glob(path+'/models_1683734698/bt-settl_m/lte*.dat') #use dat
filenames=sorted(filenames)

data_log=pd.read_csv(path+'/2411/star/stellar_simu.csv')

i=0

for filename in filenames[0:2]:
    
    data_log_p=pd.read_csv(path+'/2411/planet/%s/planet_simu.csv'%filename[-34:-23])
    
    p=analysis.planet_para(disper_planet=data_log_p, disper_star=data_log)
    p2s_in, p2s_pl, plot_in, plot_pl=p.p_2_s(order_num=[29, 28, 27, 26, 25, 24, 23], planet_posi=po)
    
    plot_in.savefig(path+'/2411/planet/%s/contrast_intrinsic.png'%filename[-34:-23], dpi=200)
    plot_pl.savefig(path+'/2411/planet/%s/contrast_planet_posi.png'%filename[-34:-23], dpi=200)
    np.save(path+'/2411/planet/%s/contrast.npy'%filename[-34:-23], np.array((p2s_in, p2s_pl)))
    
    a=p2s_in[2046*12:2046*15]
    b=p2s_pl[2046*12:2046*15]
    cont[i]=np.array((a,b))
    i+=1
    
    print ('Combination done;%s'%filename[-34:-23])

/Users/richard/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Combination done;010-4.0-0.0


/Users/richard/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Combination done;010-4.5-0.0


In [19]:
mag_estimate

,Unnamed: 0,T,log_g,metallicity,color_hk,mag_ks_h,color_jk,mag_ks_j
0,0,1000.0,4.0,0.0,-0.092750,18.788750,0.143222,19.077778
1,1,1000.0,4.5,0.0,0.119416,18.576584,-0.038496,19.259496
2,2,1000.0,5.0,0.0,0.008550,18.687450,-0.257564,19.478564
3,3,1000.0,5.5,0.0,-0.368420,19.064420,-0.646340,19.867340
4,4,1100.0,3.5,0.0,1.490601,17.205399,3.303962,15.917038
5,5,1100.0,4.0,0.0,0.150781,18.545219,0.876857,18.344143
6,6,1100.0,4.5,0.0,-0.022730,18.718730,0.106468,19.114532
7,7,1100.0,5.0,0.0,-0.015169,18.711169,-0.102282,19.323282
8,8,1100.0,5.5,0.0,-0.283699,18.979699,-0.517270,19.738270
9,9,1200.0,3.5,0.0,1.469492,17.226508,3.301115,15.919885
